In [50]:
# jupyter notebook with an example for an interferometer calculation
import sympy as sy
from sympy import I
from IPython.display import Latex
sy.init_printing()

from Interferometry import Interferometer, Hamiltonian, Pulse, U

# define symbolic variables
m1,m2,g1,g2,k,T,m,gmag,ggrav,wtrap,gravgrad1,gravgrad2,xtrap,Twait=sy.symbols("m1,m2,g1,g2,k,T,m,g_m,g,Ω,γ1,γ2,x_\Omega,T_w")



Define Hamiltonians by passing a parameter list [a,b,c,d,e,f]
 corresponding to the Hamiltonian
 $H=a p^2 + b p +c(px+xp) + d x + e + f x^2$ 
 

In [49]:
H=Hamiltonian([1/(2*m), 0,0,m*ggrav+m*gmag-m*wtrap**2*xtrap,m/2*wtrap**2 *xtrap**2,m/2*wtrap**2])
#H=Hamiltonian([1/(2*m), 0,0,m*ggrav,0,0])
IdGen =Hamiltonian([0,0,0,0,0,0])
print('Hamiltonian is:')         
print(H)
print(IdGen)

Hamiltonian is:


Define upper and lower sequence (note the ordering of operators is the same as if acting on a initial ket state)

In [51]:
upperSequenceA=[U(H,T),  Pulse(-k), U(H,T), Pulse(k),  U(H,Twait)] 
lowerSequenceA=[Pulse(-k),  U(H,T),  Pulse(k), U(H,T), U(H,Twait)]
upperSequenceB=[U(IdGen,0), U(H,T), Pulse(-k), U(H,T),U(IdGen,0),U(H,Twait),Pulse(k)] 
lowerSequenceB=[Pulse(-k),U(H,T),  Pulse(k),  U(H,T), Pulse(-k), U(H,Twait),Pulse(k)]

Initialize an interferometer object with the upper and lower sequence

In [ ]:
IA=Interferometer(upperSequenceA, lowerSequenceA, 8)
IB=Interferometer(upperSequenceB, lowerSequenceB, 8)
#calculate overlap operator
resdictA,resA=IA.overlap()
resdictB,resB=IB.overlap()

In [ ]:
# Clean up and expand everything in interferometer duration
expansionVariables = [T,Twait,wtrap]
expansionOrders = [4,4,4]
expansionPars = list(zip(expansionVariables,expansionOrders))
expansionFunsA = [resA.a, resA.b, resA.c, resA.d, resA.e, resA.f]
expansionFunsB = [resB.a, resB.b, resB.c, resB.d, resB.e, resB.f]

def expandResults(fun, expansionPars):
    tmpfun=type(fun)
    
    for item in expansionPars:
        tmpfun = sy.expand(sy.series(fun,item[0],0,n=item[1])).removeO()

    return tmpfun
    
resultA = [expandResults(item,expansionPars) for item in expansionFunsA]
resultB = [expandResults(item,expansionPars) for item in expansionFunsB]

In [ ]:
Latex(f"""
\\begin{{align}}
    &\\hat{{p}}^2 & {{{sy.latex(resultA[0])}}} \\\\
    &\\hat{{p}} & {{{sy.latex(resultA[1])}}} \\\\
    &\\hat{{p}}\\hat{{x}}+\\hat{{x}}\\hat{{p}} & {{{sy.latex(resultA[2])}}} \\\\
    &\\hat{{x}} & {{{sy.latex(resultA[3])}}} \\\\
    &\\hat{{x}}^2 & {{{sy.latex(resultA[5])}}}\\\\
    &\\hat{{1}} & {{{sy.latex(resultA[4])}}}
\\end{{align}}
""")

In [43]:
Latex(f"""
\\begin{{align}}
    &\\hat{{p}}^2 & {{{sy.latex(resultB[0])}}} \\\\
    &\\hat{{p}} & {{{sy.latex(resultB[1])}}} \\\\
    &\\hat{{p}}\\hat{{x}}+\\hat{{x}}\\hat{{p}} & {{{sy.latex(resultB[2])}}} \\\\
    &\\hat{{x}} & {{{sy.latex(resultB[3])}}} \\\\
    &\\hat{{x}}^2 & {{{sy.latex(resultB[5])}}}\\\\
    &\\hat{{1}} & {{{sy.latex(resultB[4])}}}
\\end{{align}}
""")

<IPython.core.display.Latex object>

In [44]:
spA=resultA[1]*sy.symbols('\hat{p}')
sxA=resultA[3]*sy.symbols('\hat{x}')
sphiA=resultA[4]
# Show me the operator part
display(sy.expand(spA+sxA))
# Show me the non-operator part
display(sphiA)

In [45]:
spB=resultB[1]*sy.symbols('\hat{p}')
sxB=resultB[3]*sy.symbols('\hat{x}')
sphiB=resultB[4]
# Show me the operator part
display(sy.expand(spB+sxB))
# Show me the non-operator part
display(sphiB)

In [37]:
sy.expand(spB+sxB)+sy.expand(spA+sxA)

In [36]:
display(sphiB+sphiA)

In [ ]:
# Just gimme the result for comparison with Matthias's expressions
from sympy import I

# Define magnetic gradients
mub=sy.symbols("\mu_{B}")
gfj=sy.symbols("g_{F_j}") 
mfj=sy.symbols("m_{F_j}")
x = sy.symbols("x")
Bx=sy.Function('B_x')(x)
r=sy.Mul(sy.Mul(sy.Mul(mub,gfj),mfj),sy.diff(Bx,x)).subs(x,"x_0")

In [ ]:
# Give me the result for vanishing state dependent accelerations
print('No state-dependent linear magnetic potential')
display(sy.collect(sy.expand((sp+sx+sphi)/I).subs(gmag,0),k*wtrap**2*T**2))

# Give me the result for non-vanishing state dependent accelerations (substituted and without)
print('Only state-dependent linear magnetic potential')
display(sy.collect(sy.expand((sp+sx+sphi)/I),k*wtrap**2*T**2).subs(ggrav,0))
print('Substituted definition of state-dependent linear potential')
display(sy.collect(sy.expand((sp+sx+sphi)/I).subs(gmag,r/m),k*wtrap**2*T**2))